In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Grounding service with Vertex AI

## Configuration
### Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest


In [1]:
%pip install --upgrade --quiet --user google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.3 MB/s eta 0:00:00


### Authentication to access to the GCP
* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [2]:
import sys
from IPython.display import Markdown, display

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

!gcloud config set project ai-hangsik

Updated property [core/project].


## Set the environment on GCP Project


In [5]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash-002" # @param {type:"string"}

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [8]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Tool,
)

from vertexai.preview.generative_models import grounding

vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel(MODEL_NAME)

## Response without grounding services.
This response is information from LLM without grounding services to enrich contexts sent to LLM.

##Configuration Vertex AI Search

* gs://cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs/


In [13]:
DATA_STORE_ID = "alphabet-contracts-id2"
VAIS_LOCATION = "global"

tool = Tool.from_retrieval(
    grounding.Retrieval(
        grounding.VertexAISearch(
            datastore=DATA_STORE_ID,
            project=PROJECT_ID,
            location=VAIS_LOCATION,
        )
    )
)

prompt ="2023년 Google Cloud 매출은 얼마였습니까? 분기당 정보를 알려주세요."

response = model.generate_content(
    prompt,
    tools=[tool],
    generation_config=GenerationConfig(
        temperature=0.0,
    ),
)

display(Markdown(response.text))


2023년 Google Cloud 매출은 다음과 같습니다 (단위: 백만 달러).

* **1분기 (3월 31일 종료):** 7,454백만 달러
* **2분기 (6월 30일 종료):** 8,031백만 달러
* **3분기 (9월 30일 종료):** 8,411백만 달러
* **4분기 (12월 31일 종료):** 9,192백만 달러

전체적으로 2023년 Google Cloud 매출은 33,087백만 달러 (약 330억 8,700만 달러) 증가했습니다.  이는 2022년 대비 68억 달러 증가한 수치입니다.  Google Cloud Platform과 Google Workspace가 성장을 주도했습니다.  특히 Google Cloud Platform 내에서는 인프라 및 플랫폼 서비스가 가장 큰 성장 동력이었습니다.
